In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from IPython.display import IFrame
from IPython.display import HTML

IFrame(src='./data_table_description.html', width=800, height=400)


pd.options.display.max_columns = None

In [20]:
os.listdir('./datas')

['Regular_Season_Batter_Day_by_Day_b4.csv',
 'Regular_Season_Batter.csv',
 'submission.csv',
 'Pre_Season_Batter.csv']

# 데이터 살펴보기

In [22]:
# HTML 파일 내용 읽어오기
with open('./data_table_description.html', 'r') as f:
    html_content = f.read()

# HTML 내용 출력
print('데이터 도메인')
HTML(html_content)

데이터 도메인


Column Name,Pre/Regular_Season_Batter,Regular_Season_Batter_Day_by_Day,Submission
batter_id,타자 고유 ID,타자 ID,선수 고유 ID
batter_name,타자 이름,타자 이름,선수 이름
year,년도,년도,
team,소속 팀,,
avg,타율,,
avg1,,해당 경기 타율,
avg2,,시즌 누적 타율,
G,경기,,
AB,타수,타수,
R,득점,득점,


In [12]:
pd.read_csv('./datas/Regular_Season_Batter_Day_by_Day_b4.csv').head()

,batter_id,batter_name,date,opposing_team,avg1,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,HBP,SO,GDP,avg2,year
0,0,가르시아,3.24,NC,0.333,3,1,1,0,0,0,0,0,0,1,0,1,0,0.333,2018
1,0,가르시아,3.25,NC,0.000,4,0,0,0,0,0,0,0,0,0,0,1,0,0.143,2018
2,0,가르시아,3.27,넥센,0.200,5,0,1,0,0,0,0,0,0,0,0,0,0,0.167,2018
3,0,가르시아,3.28,넥센,0.200,5,1,1,0,0,0,1,0,0,0,0,0,0,0.176,2018
4,0,가르시아,3.29,넥센,0.250,4,0,1,0,0,0,3,0,0,0,0,0,1,0.190,2018


In [13]:
pd.read_csv('./datas/Pre_Season_Batter.csv').head(1)

,batter_id,batter_name,year,team,avg,G,AB,R,H,2B,3B,HR,TB,RBI,SB,CS,BB,HBP,SO,GDP,SLG,OBP,E,height/weight,year_born,position,career,starting_salary,OPS
0,0,가르시아,2018,LG,0.350,7,20,1,7,1,0,1,11,5,0,0,2,0,3,1,0.55,0.409,1,177cm/93kg,1985년 04월 12일,내야수(우투우타),쿠바 Ciego de Avila Maximo Gomez Baez(대),NaN,0.959


In [14]:
pd.read_csv('./datas/Regular_Season_Batter.csv').head(1)

,batter_id,batter_name,year,team,avg,G,AB,R,H,2B,3B,HR,TB,RBI,SB,CS,BB,HBP,SO,GDP,SLG,OBP,E,height/weight,year_born,position,career,starting_salary,OPS
0,0,가르시아,2018,LG,0.339,50,183,27,62,9,0,8,95,34,5,0,9,8,25,3,0.519,0.383,9,177cm/93kg,1985년 04월 12일,내야수(우투우타),쿠바 Ciego de Avila Maximo Gomez Baez(대),NaN,0.902


In [15]:
pd.read_csv('./datas/submission.csv').head()

,batter_id,batter_name
0,1,강경학
1,2,강구성
2,3,강민국
3,4,강민호
4,5,강백호
